In [ ]:
from pathlib import Path
import sys

import torch

sys.path.append('/home/ovoinov/work/sk3d/dev.sk_robot_rgbd_data/src')
from skrgbd.calibration.camera_models.rv_camera_model import RVCameraModel
from skrgbd.calibration.camera_models import save_to_pt
from skrgbd.data.dataset.scene_paths import ScenePaths

The RangeVision calibration of SL camera is the same for all scans, so we only fit the inverse model once.

In [ ]:
scene_name = 'dragon'
cam_scan_i = 0
cam_id = 'b'

scene_paths = ScenePaths(scene_name, raw_dir='/mnt/datasets/sk3d/raw',
                         raw_calib_dir='/mnt/datasets/sk3d/calibration/raw_calibration/small_scale_sphere')

sl_cam_model = f'{scene_paths.sl_raw()}/scan_res_{cam_scan_i:04}/Raw/impar{cam_id}01.txt'
sl_cam_model = RVCameraModel.from_impar_txt(sl_cam_model).to('cuda', torch.double)
sl_cam_model.fit_projection_parameters(show_progress=True)
sl_cam_model = sl_cam_model.to('cpu')

In [ ]:
cam = {'a': 'stl_left', 'b': 'stl_right'}[cam_id]
cam_dict_pt = scene_paths.cam_model(cam, 'rgb', 'pt')

Path(cam_dict_pt).parent.mkdir(parents=True, exist_ok=True)
save_to_pt(cam_dict_pt, sl_cam_model)